In [1]:
import pickle

import matplotlib
matplotlib.use('Agg')
import os
import datetime
import numpy as np
import dill as pickle
import random
import sys
np.random.seed(0)
random.seed(0)
now = datetime.datetime.now().strftime("%B_%d_%Y_%H_%M_%S")
workingdirectory = os.popen('git rev-parse --show-toplevel').read()[:-1]
sys.path.append(workingdirectory)
os.chdir(workingdirectory)
#print(os.getcwd())
from codes.experimentclasses.RigidEthanolPCA import RigidEthanolPCA
from codes.otherfunctions.multirun import get_coeffs_reps_tangent
from codes.otherfunctions.multirun import get_grads_reps_pca2_tangent
from codes.otherfunctions.multiplot import plot_reg_path_ax_lambdasearch_tangent
from codes.otherfunctions.get_dictionaries import get_atoms_4
from codes.flasso.Replicate import Replicate
from codes.otherfunctions.get_grads_tangent import get_grads_tangent
from codes.otherfunctions.multirun import get_support_recovery_lambda
from codes.otherfunctions.multirun import get_lower_interesting_lambda
import matplotlib.pyplot as plt
from codes.otherfunctions.multirun import get_coeffs_and_lambdas
from codes.geometer.RiemannianManifold import RiemannianManifold
from collections import Counter

/Users/samsonkoelle/manifoldflasso_jmlr


In [2]:
import matplotlib
matplotlib.use('Agg')
import os
import datetime
import numpy as np
import dill as pickle
import random
import sys
import seaborn as sns
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib import rcParams
from collections import OrderedDict
import math
from matplotlib.lines import Line2D
from pylab import rcParams
from collections import Counter
from itertools import combinations

from shutil import copyfile
rcParams['figure.figsize'] = 25, 10

np.random.seed(0)
random.seed(0)
now = datetime.datetime.now().strftime("%B_%d_%Y_%H_%M_%S")
workingdirectory = os.popen('git rev-parse --show-toplevel').read()[:-1]
sys.path.append(workingdirectory)
os.chdir(workingdirectory)
from codes.experimentclasses.RigidEthanolPCA import RigidEthanolPCA
from codes.otherfunctions.get_dictionaries import get_all_atoms_4
from codes.otherfunctions.get_grads import get_grads
from codes.otherfunctions.multirun import get_support_recovery_lambda
from codes.otherfunctions.multirun import get_lower_interesting_lambda
from codes.otherfunctions.multirun import get_coeffs_and_lambdas
from codes.otherfunctions.multirun import get_support
from codes.otherfunctions.multiplot import plot_support_2d
from codes.otherfunctions.multiplot import plot_reg_path_ax_lambdasearch
from codes.otherfunctions.multiplot import plot_gs_v_dgnorm
from codes.otherfunctions.multiplot import plot_dot_distributions
from codes.otherfunctions.multirun import get_cosines
from codes.flasso.Replicate import Replicate
from codes.otherfunctions.multirun import get_olsnorm_and_supportsbrute
from codes.otherfunctions.multiplot import highlight_cell

#set parameters
n = 10000 #number of data points to simulate
nsel = 100 #number of points to analyze with lasso
itermax = 1000 #maximum iterations per lasso run
tol = 1e-10 #convergence criteria for lasso
#lambdas = np.asarray([0,.01,.1,1,10,100], dtype = np.float16)#lambda values for lasso
lambdas = np.asarray(np.hstack([np.asarray([0]),np.logspace(-3,1,11)]), dtype = np.float16)
n_neighbors = 1000 #number of neighbors in megaman
m = 3 #number of embedding dimensions (diffusion maps)
#diffusion_time = 1. #diffusion time controls gaussian kernel radius per gradients paper
diffusion_time = 0.05 #(yuchia suggestion)
dim = 2 #manifold dimension
dimnoise = 2
natoms = 9
cores = 3 #number of cores for parallel processing
cor = 0.0 #correlation for noise
var = 0.00001 #variance scaler for noise
cor = 0.0 #correlation for noise
var = 0.00001 #variance scaler for noise
ii = np.asarray([0,0,0,0,1,1,1,2]) # atom adjacencies for dihedral angle computation
jj = np.asarray([1,2,3,4,5,6,7,8])

#run experiment
atoms4 = np.asarray([[6,1,0,4],[4,0,2,8],[7,6,5,1],[3,0,2,4]],dtype = int)
nreps = 25
lambda_max = 1
max_search = 30

folder = workingdirectory + '/Figures/rigidethanol/' + now + 'n' + str(n) + 'nsel' + str(nsel) + 'nreps' + str(nreps)
os.mkdir(folder)

#src = workingdirectory + '/codes/experiments/rigidethanol_110120_nsel100_nreps25_var0.py'
#filenamescript = folder + '/script.py'
#copyfile(src, filenamescript)

new_MN = True
new_grad = True
# savename = 'rigidethanol_110120_alltorsions'
# savefolder = 'rigidethanol'
# loadfolder = 'rigidethanol'
# loadname = 'rigidethanol_110120_alltorsions'
if new_MN == True:
    experiment = RigidEthanolPCA(dim, cor, var, ii, jj, cores, False, atoms4)
    experiment.M, experiment.Mpca, projector = experiment.generate_data(noise=False)
    experiment.q = m
    experiment.m = m
    experiment.dimnoise = dimnoise
    experiment.projector = projector
    experiment.Mpca.geom = experiment.Mpca.compute_geom(diffusion_time, n_neighbors)
    experiment.N = experiment.Mpca.get_embedding3(experiment.Mpca.geom, m, diffusion_time, dim)
    # with open(workingdirectory + '/untracked_data/embeddings/' + savefolder + '/' + savename + '.pkl' ,
    #          'wb') as output:
    #      pickle.dump(experiment, output, pickle.HIGHEST_PROTOCOL)

atoms4,p = get_atoms_4(natoms,ii,jj)
experiment.p = p
experiment.atoms4 = atoms4
experiment.itermax = itermax
experiment.tol = tol
experiment.dnoise = dim
experiment.nreps = nreps
experiment.nsel = nsel
experiment.folder = folder


In [3]:
experiment

In [4]:
from codes.geometer.RiemannianManifold import RiemannianManifold
from codes.geometer.ShapeSpace import ShapeSpace
from codes.geometer.TangentBundle import TangentBundle
import numpy as np
import autograd.numpy as np
from codes.flasso.FlassoExperiment import FlassoExperiment
from sklearn.linear_model import LinearRegression


In [5]:
cores = 1
dim = 2
dimnoise = 2
q = 3
selected_points_save = np.zeros((nreps,nsel))
replicates = {}
nsel = 1
for i in range(nreps):
    print(i)
    selected_points = np.random.choice(list(range(n)),nsel,replace = False)
    selected_points_save[i] = selected_points
    replicates[i] = Replicate()
    replicates[i].nsel = nsel
    replicates[i].selected_points = selected_points
    #replicates[i].df_M,replicates[i].df_M_notan = get_grads(cores, dim, dimnoise, q, experiment.Mpca, experiment.M, experiment.N, selected_points)
#     replicates[i].xtrain, replicates[i].groups = experiment.construct_X_js(replicates[i].dg_M)
#     replicates[i].ytrain = experiment.construct_Y_js(replicates[i].df_M,dimnoise)
#     replicates[i].coeff_dict = {}
#     replicates[i].coeff_dict[0] = experiment.get_betas_spam2(replicates[i].xtrain, replicates[i].ytrain, replicates[i].groups, np.asarray([0]), nsel, experiment.m, itermax, tol)
#     replicates[i].combined_norms = {}
#     replicates[i].combined_norms[0] = np.linalg.norm(np.linalg.norm(replicates[i].coeff_dict[0][:, :, :, :], axis=2), axis=1)[0,:]
#     replicates[i].higher_lambda,replicates[i].coeff_dict,replicates[i].combined_norms = get_support_recovery_lambda(experiment, replicates[i],  lambda_max, max_search,dim)
#     replicates[i].lower_lambda,replicates[i].coeff_dict,replicates[i].combined_norms = get_lower_interesting_lambda(experiment, replicates[i],  lambda_max, max_search)
    #= experiment.get_betas_spam2(replicates[i].xtrain, replicates[i].ytrain, replicates[i].groups, lambdas, len(selected_points), n_embedding_coordinates, itermax, tol)


0


TypeError: get_grads() takes 5 positional arguments but 8 were given

In [6]:
replicates[i].df_M[0],replicates[i].df_M_notan[0]

AttributeError: 'Replicate' object has no attribute 'df_M'

In [7]:
replicates[i].df_M[1],replicates[i].df_M_notan[1]

AttributeError: 'Replicate' object has no attribute 'df_M'

In [29]:
def get_grads(cores, dim, dimnoise, q, Mpca, Mangles, N, selected_points):
    #dimnoise = experiment.dimnoise
    #dim = experiment.dim
    #cores = experiment.cores

    tangent_bases = Mpca.get_wlpca_tangent_sel(Mpca, selected_points, dimnoise)
    subM = RiemannianManifold(Mpca.data[selected_points], dim)
    subM.tb = TangentBundle(subM, tangent_bases)
    N.tangent_bundle = TangentBundle(N, np.swapaxes(N.geom.rmetric.Hvv[:,:dim,:],1,2))
    
    df_M = get_dF_js_idM(q, Mpca, N, subM.tb, N.tangent_bundle, selected_points, dimnoise)
    df_M2 = df_M / np.sum(np.linalg.norm(df_M, axis=1) ** 2, axis=0)
    
    df_M_notan = get_dF_js_idM_notan(q, Mpca, N, subM.tb, N.tangent_bundle, selected_points, dimnoise)
    df_M2_notan = df_M_notan / np.sum(np.linalg.norm(df_M_notan, axis=1) ** 2, axis=0)
#     dg_x = experiment.get_dx_g_full(Mangles.data[selected_points])

#     W = ShapeSpace(experiment.positions, Mangles.data)
#     dw = W.get_dw(cores, experiment.atoms3, experiment.natoms, selected_points)
#     dg_w = experiment.project(np.swapaxes(dw, 1, 2),
#                               experiment.project(dw, dg_x))

#     dg_w_pca = np.asarray([np.matmul(experiment.projector, dg_w[j].transpose()).transpose() for j in range(len(selected_points))])
#     dgw_norm = experiment.normalize(dg_w_pca)
#     dg_M = experiment.project(subM.tb.tangent_bases, dgw_norm)
    return (df_M2, df_M2_notan)


In [8]:
    def get_dF_js_idM(q, M, N, M_tangent_bundle_sub, N_tangent_bundle, selectedpoints, dim = None):

        affinity_matrix = M.geom.affinity_matrix

        nsel = len(selectedpoints)
        dF = np.zeros((nsel, dim, q))

        for i in range(nsel):
            pt = selectedpoints[i]
            neighborspt = affinity_matrix[selectedpoints[i]].indices
            deltap0 = M.data[neighborspt, :] - M.data[pt, :]
            deltaq0 = N.data[neighborspt, :] - N.data[pt, :]
            projected_M = np.matmul(M_tangent_bundle_sub.tangent_bases[i, :, :].transpose(),
                                    deltap0.transpose()).transpose()
            # projected_rescaled_M = np.matmul(np.diag(M_tangent_bundle_sub.rmetric.Gsvals[selectedpoints[i]]),projected_M.transpose())
            # projected_rescaled_M = projected_M.transpose()
            # b = np.linalg.pinv(projected_rescaled_M)
            # a = np.zeros((len(neighborspt), q))
            # rescaled_basis = np.matmul(N_tangent_bundle.tangent_bases[selectedpoints[i], :, :][:, :],
            #                            np.diag(N.geom.rmetric.Gsvals[selectedpoints[i]][:dim]))
            tan_proj = np.dot(N_tangent_bundle.tangent_bases[selectedpoints[i]],N_tangent_bundle.tangent_bases[selectedpoints[i]].transpose())
            projected_N = np.dot(deltaq0, tan_proj)
            # projected_N_expanded = np.matmul(N_tangent_bundle.tangent_bases[selectedpoints[i], :, :][:, :], projected_N)
            # a = projected_N_expanded

            # dF[i, :, :][:, :] = np.matmul(a, b).transpose()
            # dF[i, :, :][:, :] = np.linalg.lstsq(projected_M, deltaq0)[0]

            lr = LinearRegression()
            weights = affinity_matrix[selectedpoints[i]].data
            #lr.fit(projected_M, deltaq0, weights)
            lr.fit(projected_M, projected_N, weights)
            dF[i, :, :][:, :] = lr.coef_.transpose()#np.linalg.lstsq(projected_M, deltaq0)[0]#np.matmul(a, b).transpose()
        return (dF)



In [9]:

def get_dF_js_idM_notan(q, M, N, M_tangent_bundle_sub, N_tangent_bundle, selectedpoints, dim = None):

#     self = experiment
#     if dim == None:
#         dim = self.dim
#     q = self.q
    affinity_matrix = M.geom.affinity_matrix

    nsel = len(selectedpoints)
    dF = np.zeros((nsel, dim, q))

    for i in range(nsel):
        pt = selectedpoints[i]
        neighborspt = affinity_matrix[selectedpoints[i]].indices
        deltap0 = M.data[neighborspt, :] - M.data[pt, :]
        deltaq0 = N.data[neighborspt, :] - N.data[pt, :]
        projected_M = np.matmul(M_tangent_bundle_sub.tangent_bases[i, :, :].transpose(),
                                deltap0.transpose()).transpose()
#            tan_proj = np.dot(N_tangent_bundle.tangent_bases[selectedpoints[i]],N_tangent_bundle.tangent_bases[selectedpoints[i]].transpose())
        projected_N = deltaq0#np.dot(deltaq0, tan_proj)
        # projected_N_expanded = np.matmul(N_tangent_bundle.tangent_bases[selectedpoints[i], :, :][:, :], projected_N)
        # a = projected_N_expanded

        # dF[i, :, :][:, :] = np.matmul(a, b).transpose()
        # dF[i, :, :][:, :] = np.linalg.lstsq(projected_M, deltaq0)[0]

        lr = LinearRegression()
        weights = affinity_matrix[selectedpoints[i]].data
        #lr.fit(projected_M, deltaq0, weights)
        lr.fit(projected_M, projected_N, weights)
        dF[i, :, :][:, :] = lr.coef_.transpose()#np.linalg.lstsq(projected_M, deltaq0)[0]#np.matmul(a, b).transpose()
    return (dF)

In [21]:

def get_dF_js_idM_sometan(q, M, N, M_tangent_bundle_sub, N_tangent_bundle, selectedpoints, dim = None):

#     self = experiment
#     if dim == None:
#         dim = self.dim
#     q = self.q
    affinity_matrix = M.geom.affinity_matrix

    nsel = len(selectedpoints)
    dF = np.zeros((nsel, dim, q))

    for i in range(nsel):
        pt = selectedpoints[i]
        neighborspt = affinity_matrix[selectedpoints[i]].indices
        deltap0 = M.data[neighborspt, :] - M.data[pt, :]
        deltaq0 = N.data[neighborspt, :] - N.data[pt, :]
        projected_M = np.matmul(M_tangent_bundle_sub.tangent_bases[i, :, :].transpose(),
                                deltap0.transpose()).transpose()
#            tan_proj = np.dot(N_tangent_bundle.tangent_bases[selectedpoints[i]],N_tangent_bundle.tangent_bases[selectedpoints[i]].transpose())
        projected_N = deltaq0 @ N_tangent_bundle.tangent_bases[selectedpoints[i]]#np.dot(deltaq0, tan_proj)
        #delta
        lr = LinearRegression()
        weights = affinity_matrix[selectedpoints[i]].data
        #lr.fit(projected_M, deltaq0, weights)
        lr.fit(projected_M, projected_N, weights)
        dF[i, :, :][:, :] = lr.coef_.transpose()#np.linalg.lstsq(projected_M, deltaq0)[0]#np.matmul(a, b).transpose()
    return (dF)

In [ ]:
cores, dim, dimnoise, q, Mpca, Mangles, N, selected_points

In [12]:
M = experiment.M
Mpca = experiment.Mpca
Mangles = experiment.M
dim = 2
dimnoise = 2
N = experiment.N
selected_points = np.asarray([0])

#N=  experiment.N
#get_dF_js_idM(q, M, N, M_tangent_bundle_sub, N_tangent_bundle, selectedpoints, dim = None)

In [22]:
    tangent_bases = Mpca.get_wlpca_tangent_sel(Mpca, selected_points, dimnoise)
    subM = RiemannianManifold(Mpca.data[selected_points], dim)
    subM.tb = TangentBundle(subM, tangent_bases)
    N.tangent_bundle = TangentBundle(N, np.swapaxes(N.geom.rmetric.Hvv[:,:dim,:],1,2))
    
    df_M = get_dF_js_idM(q, Mpca, N, subM.tb, N.tangent_bundle, selected_points, dimnoise)
    df_M2 = df_M / np.sum(np.linalg.norm(df_M, axis=1) ** 2, axis=0)
    
    df_M_notan = get_dF_js_idM_notan(q, Mpca, N, subM.tb, N.tangent_bundle, selected_points, dimnoise)
    df_M2_notan = df_M_notan / np.sum(np.linalg.norm(df_M_notan, axis=1) ** 2, axis=0)
    
    df_M_sometan = get_dF_js_idM_sometan(q, Mpca, N, subM.tb, N.tangent_bundle, selected_points, dimnoise)
    df_M2_sometan = df_M_sometan / np.sum(np.linalg.norm(df_M_sometan, axis=1) ** 2, axis=0)   

ValueError: could not broadcast input array from shape (2,2) into shape (2,3)

In [16]:
df_M2

array([[[ -2.68248629, -26.42504771, -28.71425395],
        [ -0.06996217,   4.27826027,  -2.27134559]]])

In [15]:
df_M2_notan

array([[[ -2.68185082, -27.35185276, -13.51148841],
        [ -0.0407752 ,  -2.28008977, -15.28539446]]])

In [37]:
    M_tangent_bundle_sub = subM.tb
    N_tangent_bundle = N.tangent_bundle
    
    affinity_matrix = Mpca.geom.affinity_matrix

    nsel = len(selected_points)
    dF = np.zeros((nsel, dim, q))

    for i in range(nsel):
        pt = selected_points[i]
        neighborspt = affinity_matrix[selected_points[i]].indices
        deltap0 = Mpca.data[neighborspt, :] - Mpca.data[pt, :]
        deltaq0 = N.data[neighborspt, :] - N.data[pt, :]
        projected_M = np.matmul(M_tangent_bundle_sub.tangent_bases[i, :, :].transpose(),
                                deltap0.transpose()).transpose()
#            tan_proj = np.dot(N_tangent_bundle.tangent_bases[selectedpoints[i]],N_tangent_bundle.tangent_bases[selectedpoints[i]].transpose())
        projected_N = deltaq0 @ N_tangent_bundle.tangent_bases[selected_points[i]]#np.dot(deltaq0, tan_proj)
        #delta
        lr = LinearRegression()
        weights = affinity_matrix[selected_points[i]].data
        #lr.fit(projected_M, deltaq0, weights)
        lr.fit(projected_M, projected_N, weights)
        dF[i, :, :][:, :] = lr.coef_.transpose()#np.linalg.lstsq(projected_M, deltaq0)[0]#np.matmul(a, b).transpose()


ValueError: could not broadcast input array from shape (2,2) into shape (2,3)

In [41]:
projected_N.shape

(115, 2)

In [40]:
lr.coef_.transpose()

array([[ 0.37590285, -0.00609069],
       [ 0.00917712, -0.00730227]])

In [39]:
deltaq0.shape

(115, 3)

In [36]:
N_tangent_bundle.tangent_bases[selected_points[i]].shape

(3, 2)

In [35]:
dF.shape

(1, 2, 3)

In [34]:
lr.coef_.transpose()

array([[ 0.37590285, -0.00609069],
       [ 0.00917712, -0.00730227]])

In [31]:
lr.coef_.transpose()

array([[ 0.37590285, -0.00609069],
       [ 0.00917712, -0.00730227]])

In [ ]:
#The true champion

import autograd.numpy as np
from autograd import jacobian
from autograd import elementwise_grad
from autograd import grad

import logging
from copy import deepcopy

#import numpy as np
from scipy.special import expit
from pyglmnet import utils


class GLM:

    def __init__(self, xs, ys, reg_lambda, group,max_iter, learning_rate, tol,parameter):
        self.xs = xs
        self.ys = ys
        self.reg_lambda = reg_lambda
        self.group = np.asarray(group)
        #print(self.group.shape)
        self.max_iter = max_iter
        self.learning_rate = learning_rate
        self.tol = tol
        self.Tau = None
        self.alpha = 1.
        self.lossresults = {}
        self.dls = {}
        self.parameter = parameter
        self.l2loss = {}
        self.penalty = {}

    def _prox(self,beta, thresh):
        """Proximal operator."""

        #print(thresh, beta)
        #print('beginprox', beta[0:2],thresh)
        group_ids = np.unique(self.group)
        result = np.zeros(beta.shape)
        result = np.asarray(result, dtype = float)
        #print('gids',group_ids)
        for i in range(len(group_ids)):
            gid = i
            #print(self.group)
            idxs_to_update = np.where(self.group == gid)[0]
            #print('idx',idxs_to_update)
            #print('norm', np.linalg.norm(beta[idxs_to_update]))
            if np.linalg.norm(beta[idxs_to_update]) > 0.:
                #print('in here', len(idxs_to_update))
                potentialoutput = beta[idxs_to_update] - (thresh / np.linalg.norm(beta[idxs_to_update])) * beta[idxs_to_update]
                posind = np.where(beta[idxs_to_update] > 0.)[0]
                negind = np.where(beta[idxs_to_update] < 0.)[0]
                po = beta[idxs_to_update].copy()
                #print('potention', potentialoutput[0:2])
                po[posind] = np.asarray(np.clip(potentialoutput[posind],a_min = 0., a_max = 1e15), dtype = float)
                po[negind] = np.asarray(np.clip(potentialoutput[negind],a_min = -1e15, a_max = 0.), dtype = float)
                result[idxs_to_update] = po
        #print('end', result[0:2])
        return result

    def _grad_L2loss(self, beta, Xs, ys):
        #print(beta.shape,X.shape,y.shape)
        i = 0
        for x,y in zip(Xs,ys):
            if y.ndim == 1:
                y = y[:, np.newaxis]
            n_samples = np.float(X.shape[0])
            z = np.dot(X, beta)
            #grad_beta = 1. / n_samples * np.transpose(np.dot(np.transpose(z - y), X))
            grad_beta[i] = np.transpose(np.dot(np.transpose(z - y), X))
            i = i+1
        gb = np.mean(grad_beta, axis = 0)
        #print('gb',grad_beta.shape)
        return gb

    def _loss(self,beta, reg_lambda, X, y):
        """Define the objective function for elastic net."""
        L = self._logL(beta, X, y)
        P = self._L1penalty(beta)
        J = -L + reg_lambda * P
        return J

    def _logL(self,beta, X, y):
        """The log likelihood."""
        #print('beginlogL', np.linalg.norm(beta), np.linalg.norm(X), np.linalg.norm(y),y.shape,beta.shape,X.shape,)
        l = np.dot(X, beta)
        logL = -0.5 * np.sum((y - l)**2)
        #print('endlogL',logL)
        return logL

    def _L2loss(self,beta,X,y):
        #print('beginl2', np.linalg.norm(beta), np.linalg.norm(X), np.linalg.norm(y), y.shape)
        output = -self._logL(beta, X, y)
        #print('outl2',output)
        return(output)

    def _L1penalty(self, beta):
        """The L1 penalty"""
        # Compute the L1 penalty
        group = self.group
        group_ids = np.unique(self.group)
        L1penalty = 0.0
        for group_id in group_ids:
            L1penalty += np.linalg.norm(beta[self.group == group_id], 2)
        return L1penalty

    #def fhatlambda(self,lamb,x,y):
    def fhatlambda(self,lamb,betanew,betaold):
        xs = self.xs
        ys = self.ys
        #print(ys.shape,'fhatlam')
        #print(self._L2loss(betaold,xs,ys),self._L2loss(betanew,xs,ys),'old','new')
        output = self._L2loss(betaold,xs,ys) + np.dot(self._grad_L2loss(betaold,xs,ys).transpose(),(betanew-betaold)) + (1/(2*lamb)) * np.linalg.norm(betanew-betaold)**2
        return(output)

    #_btalgorithm(yk,lamb,.5,1000, rl)
    def _btalgorithm(self,bet,lam,b,maxx,rl):

        #print('lam',lam)
        X = self.xs
        y = self.ys
        #print('beginbt', np.linalg.norm(y))
        #print('beginbt',self._L2loss(bet,X,y))
        #print(np.linalg.norm(bet))
        grad_beta = self._grad_L2loss(beta = bet, X = X, y = y)
        for i in range(maxx):
            betn = bet - lam * grad_beta
            z = self._prox(betn, lam * rl)
            fz = self._L2loss(z,X,y)
            #print(fz,'fz')
            fhatz = self.fhatlambda(lam,z, bet)
            if fz <= fhatz:
            #print(fhatz - fz)
            #if 0 <= 1:
                break
            lam = b*lam
        return(z,lam)

    def fit(self):


        group  = self.group
        print(group.shape)
        lambdas = self.reg_lambda
        parameter = self.parameter
        X = self.xs
        #print(X.shape)
        y = self.ys

        np.random.RandomState(0)
        group = np.asarray(group, dtype = np.int64)

        #print(group.shape[0])
        group.dtype = np.int64
        #print(group.shape[0])
        #print(X.shape[1])
        if group.shape[0] != X.shape[1]:
            raise ValueError('group should be (n_features,)')

        # type check for data matrix
        if not isinstance(X, np.ndarray):
            raise ValueError('Input data should be of type ndarray (got %s).'
                             % type(X))

        n_features = np.float(X.shape[1])
        n_features = np.int64(n_features)
        if y.ndim == 1:
            y = y[:, np.newaxis]
            self.ys = y
        #print(y.shape)
        n_classes = 1
        n_classes = np.int64(n_classes)

        beta_hat = 1 / (n_features) * np.random.normal(0.0, 1.0, [n_features, n_classes])
        fit_params = list()

        for l, rl in enumerate(lambdas):
            fit_params.append({'beta': beta_hat})
            if l == 0:
                fit_params[-1]['beta'] = beta_hat
            else:
                fit_params[-1]['beta'] = fit_params[-2]['beta']
            tol = self.tol
            alpha = 1.
            beta = np.zeros([n_features, n_classes])
            beta = fit_params[-1]['beta']
            #print('losser',self._L2loss(beta,X,y))
            g = np.zeros([n_features, n_classes])
            L, DL ,L2,PEN = list(), list() , list(), list()
            lamb = self.learning_rate
            bm1 = beta.copy()
            bm2 = beta.copy()
            for t in range(0, self.max_iter):
                L.append(self._loss(beta, rl, X, y))
                L2.append(self._L2loss(beta,X,y))
                PEN.append(self._L1penalty(beta))
                w = (t / (t+ 3))
                yk = beta + w*(bm1 - bm2)
                #print('losser',self._L2loss(yk,X,y))
                #print('beforebt',np.linalg.norm(yk),np.linalg.norm(X),np.linalg.norm(y))
                beta , lamb = self._btalgorithm(yk,lamb,.5,1000, rl)
                #X = self.xs
                #y = self.ys
                #print('losser2',self._L2loss(beta,X,y))
                bm2 = bm1.copy()
                bm1 = beta.copy()
                if t > 1:
                    DL.append(L[-1] - L[-2])
                    if np.abs(DL[-1] / L[-1]) < tol:
                        print('converged', rl)
                        msg = ('\tConverged. Loss function:'
                               ' {0:.2f}').format(L[-1])
                        msg = ('\tdL/L: {0:.6f}\n'.format(DL[-1] / L[-1]))
                        break

            #print(beta)
            fit_params[-1]['beta'] = beta
            self.lossresults[rl] = L
            self.l2loss[rl] = L2
            self.penalty[rl] = PEN
            self.dls[rl] = DL
            #print(L)
        # Update the estimated variables

        self.fit_ = fit_params
        self.ynull_ = np.mean(y)

        # Return
        return self


In [ ]:
# import matplotlib
# matplotlib.use('Agg')
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import scipy
import matplotlib.colors
import spams
from collections import OrderedDict
from pylab import rcParams
from codes.flasso.GLMaccelerated import GLM

rcParams['figure.figsize'] = 25, 10


def cosine_similarity(a, b):
    output = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
    return (output)


class FlassoExperiment:
    """
    FlassoExperiment
    """

    def __init__(self):
        2 + 2

    def get_norms(self, differential):
        n = differential.shape[0]
        # could be p, or q
        p = differential.shape[1]
        d = differential.shape[2]

        differential_normalized = np.zeros(differential.shape)
        vectornorms = np.zeros((n, p))
        for i in range(n):
            for j in range(p):
                if np.linalg.norm(differential[i, j, :]) > 0:
                    vectornorms[i, j] = np.linalg.norm(differential[i, j, :])

        psum = np.sum(vectornorms, axis=0)
        return (psum / n)

    def _flatten_coefficient(self, coeff):
        n = coeff.shape[1]
        p = coeff.shape[2]
        q = coeff.shape[0]

        output = np.zeros((n * p * q))
        for k in range(q):
            for i in range(n):
                output[((k * n * p) + (i * p)):((k * n * p) + (i + 1) * p)] = coeff[k, i, :]
        return (output)

    def get_l2loss(self, coeffs, ys, xs):

        n = coeffs.shape[2]
        nlam = coeffs.shape[0]
        output = np.zeros(nlam)
        for i in range(nlam):
            coeffvec = self._flatten_coefficient(coeffs[i])
            output[i] = np.sum((ys - np.dot(coeffvec, xs.transpose())) ** 2)
        output = output / n
        return (output)

    def normalize(self, differential):
        n = differential.shape[0]
        # could be p, or q
        p = differential.shape[1]
        d = differential.shape[2]

        gammas = np.sum(np.sum(differential ** 2, axis=2), axis=0) ** (.5)
        normed = np.swapaxes(differential, 1, 2) / gammas
        #print(normed.shape)
        normed = np.swapaxes(normed, 1, 2)
    #
    #     differential_normalized = np.zeros(differential.shape)
    #     vectornorms = np.zeros((n, p))
    #     for i in range(n):
    #         for j in range(p):
    #             if np.linalg.norm(differential[i, j, :]) > 0:
    #                 vectornorms[i, j] = np.linalg.norm(differential[i, j, :])
    #     # psum = np.sum(vectornorms, axis = 0)
    #     psum = np.sqrt(np.sum(vectornorms ** 2, axis=0))#np.sum(vectornorms ** 2, axis=0)#
    #     for j in range(p):
    #         if psum[j] > 0:
    #             differential_normalized[:, j, :] = (differential[:, j, :] / psum[j])  # *n
    #
    #     return (differential_normalized)
        return(normed)


    def get_betas_sam(self, xtrain, ytrain, groups, lambdas, n, q, max_iter, tol, learning_rate):

        p = len(np.unique(groups))
        models = GLM(xs=xtrain, ys=ytrain,
                     tol=tol,
                     group=groups,
                     learning_rate=learning_rate,
                     max_iter=max_iter,
                     # reg_lambda=np.logspace(np.log(100), np.log(0.01), 5, base=np.exp(1)))
                     reg_lambda=lambdas,
                     parameter=.5)
        models.fit()
        nlam = len(lambdas)
        organizedbetas = np.zeros((nlam, q, n, p))
        for l in range(nlam):
            organizedbetas[l, :, :, :] = np.reshape(models.fit_[l]['beta'], (q, n, p))
        # return(models, organizedbetas)
        return (organizedbetas)

    def construct_X(self, dg_M):
        """ dg_M should have shape n x p x dim
        """
        n = dg_M.shape[0]
        dim = dg_M.shape[2]
        p = dg_M.shape[1]

        xmat = np.zeros((n * dim, n * p))
        for i in range(n):
            xmat[(i * dim):(i * dim + dim), (i * p):(i * p + p)] = dg_M[i, :, :].transpose()
        b = [xmat] * dim
        xmatq = scipy.linalg.block_diag(*b)
        groups = np.tile(np.tile(np.asarray(np.linspace(start=0, stop=(p - 1), num=p), dtype=int), n), dim)

        return (xmatq, list(groups))

    def construct_X_js(self, dg_M):
        """ dg_M should have shape n x p x dim
        """
        n = dg_M.shape[0]
        dim = dg_M.shape[2]
        p = dg_M.shape[1]
        q = self.q

        xmat = np.zeros((n * dim, n * p))
        for i in range(n):
            xmat[(i * dim):(i * dim + dim), (i * p):(i * p + p)] = dg_M[i, :, :].transpose()
        b = [xmat] * q
        xmatq = scipy.linalg.block_diag(*b)
        groups = np.zeros(n * p * q)
        groups = np.tile(np.tile(np.asarray(np.linspace(start=0, stop=(p - 1), num=p), dtype=int), n), q)

        return (xmatq, list(groups))

    def construct_X_js_subset(self, dg_M, selind):
        dg_M_subset = np.zeros(dg_M.shape)
        dg_M_subset[:, selind, :] = dg_M[:, selind, :]
        output = self.construct_X_js(dg_M_subset)
        return (output)

    def construct_Y(self, df_M):
        """ df_M should have shape n x dim x dim
        """
        n = df_M.shape[0]
        dim = df_M.shape[1]

        #reorg1 = np.swapaxes(df_M, 0, 1)
        yvec = np.reshape(np.swapaxes(df_M,0,1), (n * dim * dim))
        return (yvec)

    def construct_Y_js(self, df_M, dim=None):
        """ df_M should have shape n x dim x q
        """
        n = df_M.shape[0]
        q = self.q
        if dim == None:
            dim = self.dim

        reorg1 = np.swapaxes(df_M, 0, 2)
        reorg2 = np.swapaxes(reorg1, 2, 1)
        # yvec = np.reshape(reorg2, (n*dim*dim))
        yvec = np.reshape(reorg2, (n * dim * q))
        return (yvec)

    def plot_convergence(self, models, name='lossplot.pdf'):
        for key in list(models.keys()):
            # key = list(models.keys())[0]
            xval = np.log(np.asarray(list(range(len(list(models[key].lossresults.values())[0])))) + 1)
            y = list(models[key].lossresults.values())[0]
            plt.plot(xval, y)
        plt.legend(list(models.keys()), loc='upper right')
        plt.savefig(name)

    def plot_convergence_sam(self, models, name='lossplot.pdf'):
        for key in list(models.lossresults.keys()):
            # key = list(models.keys())[0]
            xval = np.log(np.asarray(list(range(len(models.lossresults[key])))) + 1)
            y = models.lossresults[key]
            plt.plot(xval, y)
        plt.legend(list(models.lossresults.keys()), loc='upper right')
        plt.savefig(name)

    def plot_bh(self, coeffs, nsample_pts, p, name='beta'):
        dim = self.dim
        # p = self.p
        n = nsample_pts
        nlam = coeffs.shape[0]

        for l in range(len(lambdas)):
            if dim > 1:
                fig, axes = plt.subplots(1, q, figsize=(15, 30))
                for k in range(q):
                    tempplot = axes[k].imshow(coeffs[l, k, :, :])
                    plt.colorbar(tempplot, ax=axes[k])
            if dim == 1:
                k = 0
                fig, ax = plt.subplots(figsize=(5, 5))
                ax.imshow(coeffs[l, k, :, :])
            fig.savefig('lambda' + name + 'heatmap.pdf')

    def compute_penalty2(self, coeffs):
        n = coeffs.shape[2]
        nlam = coeffs.shape[0]
        q = coeffs.shape[1]
        p = coeffs.shape[3]

        # p = self.p
        pen = np.zeros(nlam)
        for l in range(nlam):
            norm2 = np.zeros(p)
            for j in range(p):
                norm2[j] = np.linalg.norm(coeffs[l, :, :, j])
            pen[l] = np.sum(norm2)
        pen = pen / n
        return (pen)

    def plot_penalty(self, coeffs, xaxis, xlabel, xlog, ylog, title, filename):
        ylabel = r"$\frac{1}{n}\displaystyle \|\beta\|_{1,2}$ "

        if xlog:
            filename = filename + 'xlog'
        if ylog:
            filename = filename + 'ylog'

        pens = self.compute_penalty2(coeffs)
        fig, axes = plt.subplots(1, 1, figsize=(15, 15))
        axes.plot(xaxis, pens, 'go--', linewidth=5, markersize=0, alpha=1)
        axes.set_ylim(bottom=1e-4, top=pens.max())
        if xlog:
            axes.semilogx()
        if ylog:
            axes.semilogy()
        axes.tick_params(labelsize=50)
        fig.text(0.5, 0.04, xlabel, ha='center', va='center', fontsize=50)
        fig.text(0.05, 0.5, ylabel, ha='center', va='center', rotation='vertical', fontsize=60)
        plt.suptitle(title, fontsize=55)
        fig.savefig(filename + 'penalty' + str(n))

    def plot_predictions(self, coeffs, xs, ys, xaxis, xlabe, xlog, ylog, title, filename):
        2 + 2

    def plot_l2loss(self, coeffs, xs, ys, xaxis, xlabel, xlog, ylog, title, filename):

        ylabel = r"$\displaystyle \frac{1}{n}\|y - x\beta\|_2^2$"

        if xlog:
            filename = filename + 'xlog'
        if ylog:
            filename = filename + 'ylog'

        losses = self.get_l2loss(coeffs, ys, xs)
        fig, axes = plt.subplots(1, 1, figsize=(15, 15))
        axes.plot(xaxis, losses, 'go--', linewidth=5, markersize=0, alpha=1)
        axes.set_ylim(bottom=1e-4, top=losses.max())
        if xlog:
            axes.semilogx()
        if ylog:
            axes.semilogy()
        axes.tick_params(labelsize=50)
        fig.text(0.5, 0.04, xlabel, ha='center', va='center', fontsize=50)
        fig.text(0.05, 0.5, ylabel, ha='center', va='center', rotation='vertical', fontsize=60)
        plt.suptitle(title, fontsize=55)
        fig.savefig(filename + 'loss' + str(n))

    def get_cosines(self, dg):
        n = dg.shape[0]
        p = dg.shape[1]
        d = dg.shape[2]

        coses = np.zeros((n, p, p))
        for i in range(n):
            for j in range(p):
                for k in range(p):
                    coses[i, j, k] = cosine_similarity(dg[i, j, :], dg[i, k,
                                                                    :])  # sklearn.metrics.pairwise.cosine_similarity(X = np.reshape(dg[:,i,:], (1,d*n)),Y = np.reshape(dg[:,j,:], (1,d*n)))[0][0]
        cos_summary = np.abs(coses).sum(axis=0) / n
        return (cos_summary)

    def normalize_xaxis(self, coeff, groups):

        # groups = np.asarray(may_experiment.groups)
        tempsum = 0
        i = 0
        ngroups = coeff.shape[3]
        for j in range(ngroups):
            tempsum = tempsum + np.linalg.norm(coeff[i, :, :, j])
        beta0norm = tempsum

        xaxis = np.zeros(len(lambdas))
        for i in range(len(lambdas)):
            tempsum = 0
            for j in range(ngroups):
                tempsum = tempsum + np.linalg.norm(coeff[i, :, :, j])
            xaxis[i] = tempsum / beta0norm
        return (xaxis)

    def get_betas_spam2(self, xs, ys, groups, lambdas, n, q, itermax, tol):

        # n = xs.shape[0]
        p = len(np.unique(groups))
        lambdas = np.asarray(lambdas, dtype=np.float64)
        yadd = np.expand_dims(ys, 1)
        groups = np.asarray(groups, dtype=np.int32) + 1
        W0 = np.zeros((xs.shape[1], yadd.shape[1]), dtype=np.float32)
        Xsam = np.asfortranarray(xs, dtype=np.float32)
        Ysam = np.asfortranarray(yadd, dtype=np.float32)
        coeffs = np.zeros((len(lambdas), q, n, p))
        for i in range(len(lambdas)):
            # alpha = spams.fistaFlat(Xsam,Dsam2,alpha0sam,ind_groupsam,lambda1 = lambdas[i],mode = mode,itermax = itermax,tol = tol,numThreads = numThreads, regul = "group-lasso-l2")
            # spams.fistaFlat(Y,X,W0,TRUE,numThreads = 1,verbose = TRUE,lambda1 = 0.05, it0 = 10, max_it = 200,L0 = 0.1, tol = 1e-3, intercept = FALSE,pos = FALSE,compute_gram = TRUE, loss = 'square',regul = 'l1')
            output = spams.fistaFlat(Ysam, Xsam, W0, True, groups=groups, numThreads=-1, verbose=True,
                                     lambda1=lambdas[i], it0=100, max_it=itermax, L0=0.5, tol=tol, intercept=False,
                                     pos=False, compute_gram=True, loss='square', regul='group-lasso-l2', ista=False,
                                     subgrad=False, a=0.1, b=1000)
            coeffs[i, :, :, :] = np.reshape(output[0], (q, n, p))
            # print(output[1])
        return (coeffs)


    def get_betas_sam_list(self, xtrain, ytrain, groups, lambdas, n, q, max_iter, tol, learning_rate):

        p = len(np.unique(groups))
        models = GLM(xs=xtrain, ys=ytrain,
                     tol=tol,
                     group=groups,
                     learning_rate=learning_rate,
                     max_iter=max_iter,
                     # reg_lambda=np.logspace(np.log(100), np.log(0.01), 5, base=np.exp(1)))
                     reg_lambda=lambdas,
                     parameter=.5)
        models.fit()
        nlam = len(lambdas)
        organizedbetas = np.zeros((nlam, q, n, p))
        for l in range(nlam):
            organizedbetas[l, :, :, :] = np.reshape(models.fit_[l]['beta'], (q, n, p))
        # return(models, organizedbetas)
        return (organizedbetas)
